In [1]:
%pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.32.0 trl==0.4.7
%pip install -U datasets

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle-environments 1.14.3 requires transformers>=4.33.1, but you have transformers 4.32.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 805.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 15.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 11.0.0
    Uninstalling pyarrow-11.0.0:
      Successfully uninstalled pyarrow-11.0.0
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.12.2
    Uninstalling fsspec-2023.12.2:
      Successfully uninstalled fsspec-2023.12.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
 

In [2]:
import os, torch, logging
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

2024-02-25 20:58:47.917440: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 20:58:47.917564: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 20:58:48.035836: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Load the dataset and select part of it
data_name = "Megnis/python_code_instructions_27k_Saiga"
training_data = load_dataset(data_name, split="train")
training_data = training_data.select(range(2000))

# Model and tokenizer names
base_model_name = "pieken/saiga2_7b_lora_merged"
refined_model = "saiga2-7b-python-code"

# Load tokenizer and set it's settings
llama_tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"  # Fix for fp16

# Quantization Config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

# Load model and set it's settings
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map={"": 0}
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

Generating train split:   0%|          | 0/27224 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/744 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/118 [00:00<?, ?B/s]

...

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [4]:
# Example of generating text before tuning
system_prompt = "Ты — Сайга, русскоязычный автоматический ассистент по языку программирования Python. Ты разговариваешь с людьми и помогаешь им."
query = "Напишите функцию Python с именем «add_numbers», которая принимает два параметра: «num1» и «num2» и возвращает сумму двух чисел."
text_gen = pipeline(task="text-generation", model=base_model, tokenizer=llama_tokenizer, max_length=500)
output = text_gen(f"<s>system\n{system_prompt}</s>\n<s>user\n{query}</s>\n<s>bot")
print(output[0]['generated_text'])

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1411: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


<s>system
Ты — Сайга, русскоязычный автоматический ассистент по языку программирования Python. Ты разговариваешь с людьми и помогаешь им.</s>
<s>user
Напишите функцию Python с именем «add_numbers», которая принимает два параметра: «num1» и «num2» и возвращает сумму двух чисел.</s>
<s>bot
```python
def add_numbers(num1, num2):
    return num1 + num2
```

Эта функция принимает два параметра и возвращает сумму двух чисел. Она использует оператор `+` для сложения чисел.


In [5]:
# Setting LoRA Config
peft_parameters = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=8,
    bias="none",
    task_type="CAUSAL_LM"
)

# Setting Training Params
train_params = TrainingArguments(
    output_dir="./results_modified",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to = "none"
)

# Create Trainer
fine_tuning = SFTTrainer(
    model=base_model,
    train_dataset=training_data,
    peft_config=peft_parameters,
    dataset_text_field="text",
    tokenizer=llama_tokenizer,
    args=train_params
)

# Create Trainer
fine_tuning.train()

# Save Model
fine_tuning.model.save_pretrained(refined_model)

/opt/conda/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

You are using 8-bit optimizers with a version of `bitsandbytes` < 0.41.1. It is recommended to update your version as a major bug has been fixed in 8-bit optimizers.
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,0.895600
50,0.570600
75,0.537100
100,0.382600
125,0.457900
150,0.331300
175,0.429900
200,0.294400
225,0.395300
250,0.284400


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

In [6]:
# Load LoRA model
model = PeftModel.from_pretrained(base_model, refined_model)

In [7]:
# Generate Text after tuning
query = "Напишите функцию Python с именем «add_numbers», которая принимает два параметра: «num1» и «num2» и возвращает сумму двух чисел."
text_gen = pipeline(task="text-generation", model=model, tokenizer=llama_tokenizer, max_length=500)
output = text_gen(f"<s>system\n{system_prompt}</s>\n<s>user\n{query}</s>\n<s>bot")
print(output[0]['generated_text'])

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead',

<s>system
Ты — Сайга, русскоязычный автоматический ассистент по языку программирования Python. Ты разговариваешь с людьми и помогаешь им.</s>
<s>user
Напишите функцию Python с именем «add_numbers», которая принимает два параметра: «num1» и «num2» и возвращает сумму двух чисел.</s>
<s>bot
```python
def add_numbers(num1, num2):
    """
    This function takes two numbers as input and returns their sum.

    Parameters:
    num1 (int or float): The first number.
    num2 (int or float): The second number.

    Returns:
    int or float: The sum of num1 and num2.
    """
    return num1 + num2

# Example usage
result = add_numbers(5, 3)
print(result)  # Output: 8


``` Объяснение: - Функция «add_numbers» принимает два параметра: «num1» и «num2». - Он возвращает сумму двух чисел. - Приведен пример использования, чтобы продемонстрировать, как можно использовать эту функцию. - Функцию можно легко протестировать с использованием различных значений. 

```

```

```

```

```

```

```

```

```

In [8]:
# Generate Text after tuning
query = "Cоздайте функцию Python с именем «calculate_jaccard_similarity», которая принимает на вход два набора: set1 и set2. Функция должна вычислить сходство Жаккара между двумя наборами и вернуть результат."
text_gen = pipeline(task="text-generation", model=model, tokenizer=llama_tokenizer, max_length=500)
output = text_gen(f"<s>system\n{system_prompt}</s>\n<s>user\n{query}</s>\n<s>bot")
print(output[0]['generated_text'])

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead',

<s>system
Ты — Сайга, русскоязычный автоматический ассистент по языку программирования Python. Ты разговариваешь с людьми и помогаешь им.</s>
<s>user
Cоздайте функцию Python с именем «calculate_jaccard_similarity», которая принимает на вход два набора: set1 и set2. Функция должна вычислить сходство Жаккара между двумя наборами и вернуть результат.</s>
<s>bot
```python
def calculate_jaccard_similarity(set1, set2):
    """
    This function takes two sets as input: set1 and set2. It calculates the Jaccard similarity between the two sets and returns the result.

    Parameters:
    set1 (set): The first set.
    set2 (set): The second set.

    Returns:
    float: The Jaccard similarity between the two sets.
    """
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    jaccard_similarity = intersection / union
    return jaccard_similarity
```

 Пример использования:```python
set1 = {1, 2, 3, 4}
set2 = {2, 3, 4, 5}
jaccard_similarity = calculate_jaccard_sim

In [9]:
# Generate Text after tuning
query = "Generate a Python code for crawling a website for a specific type of data. website: www.example.com data to crawl: phone numbers"
text_gen = pipeline(task="text-generation", model=model, tokenizer=llama_tokenizer, max_length=500)
output = text_gen(f"<s>system\n{system_prompt}</s>\n<s>user\n{query}</s>\n<s>bot")
print(output[0]['generated_text'])

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead',

<s>system
Ты — Сайга, русскоязычный автоматический ассистент по языку программирования Python. Ты разговариваешь с людьми и помогаешь им.</s>
<s>user
Generate a Python code for crawling a website for a specific type of data. website: www.example.com data to crawl: phone numbers</s>
<s>bot
```python
import requests
from bs4 import BeautifulSoup

url = 'https://www.example.com'
soup = BeautifulSoup(requests.get(url).content, 'lxml')
phone_numbers = [phone_number.text for phone_number in soup.find_all('span', class_='phone-number')]
print(phone_numbers)
``` 
- Этот код загружает веб-страницу www.example.com с помощью функции requests.get(). Он использует библиотеку BeautifulSoup для разбора HTML-кода страницы. С помощью метода find_all() он ищет все элементы `span` с классом `phone-number` на странице. Наконец, он преобразует все найденные телефоны в список. Выход: `[123-456-7890, 234-567-8901, 345-678-9012]` - это все телефоны, найденные на этой веб-странице. 

```

```

```

```

```

`

In [10]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("huggingface")

In [11]:
from huggingface_hub import login
login(secret_value_0)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [12]:
# Push to huggingface hub
model.push_to_hub("Megnis/saiga2-7b-python-code")

adapter_model.bin:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Megnis/saiga2-7b-python-code/commit/a406eb16f089c34f029b6c4dcb41c8295c89ea83', commit_message='Upload model', commit_description='', oid='a406eb16f089c34f029b6c4dcb41c8295c89ea83', pr_url=None, pr_revision=None, pr_num=None)